In [ ]:
# ! python -m pip install amplpy --upgrade
# ! python -m amplpy.modules install highs scip
# ! python -m amplpy.modules activate

In [7]:
import os
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import numpy as np
from amplpy import AMPL, tools, Environment
import json

In [2]:

os.chdir("/Users/gabrielefossi/Documents/CDMO/MCP_GROUP_CDMO")

In [3]:
def parse_dat_file(filepath):
    with open(filepath, 'r') as file:
        lines = file.readlines()
        
        m = int(lines[0].strip())
        n = int(lines[1].strip())
        l_values = list(map(int, lines[2].strip().split()))
        s_values = list(map(int, lines[3].strip().split()))
        distance_matrix = [list(map(int, line.strip().split())) for line in lines[4:]]
        
        return {
            'm': m,
            'n': n,
            'l_values': l_values,
            's_values': s_values,
            'distance_matrix': distance_matrix
        }

def read_dat_files(folder_path):
    dat_files = [f for f in os.listdir(folder_path) if f.endswith('.dat')]
    instances = {}
    
    for dat_file in dat_files:
        file_path = os.path.join(folder_path, dat_file)
        instances[dat_file] = parse_dat_file(file_path)
    
    return instances

folder_path = 'MIP/Instances'  # Adjust this path as necessary
instances = read_dat_files(folder_path)

In [4]:
solvers = ['highs', 'scip']

In [5]:
instances = dict(sorted(instances.items()))

In [8]:
def run_MIP_model(instance, solver):
    ampl = AMPL(Environment())
    
    model_path = "MIP/model.mod"
    ampl.read(model_path)

    ampl.param["m"] = instance['m']
    ampl.param["n"] = instance['n']
    ampl.param['l_values'] = instance['l_values']
    ampl.param['s_values'] = instance['s_values']
    ampl.param['distance_matrix'] = np.array(instance['distance_matrix'])
    
    # Set solver and time limit
    ampl.option["solver"] = solver
    ampl.option[f"{solver}_options"] = "timelim=300"
    
    # Solve the model
    ampl.solve()
    
    # Check if the solution is optimal
    is_solved = ampl.get_value("solve_result")
    obj_value = int(ampl.get_objective('obj_max_distance').value())
    solve_time = int(ampl.get_value('_total_solve_time'))
    solver_status = ampl.get_value('solve_message')
    
    # Determine if the solution is optimal
    if 'optimal solution' in solver_status:
        is_optimal = 'true'
    else:
        is_optimal = 'false'
    if 'infeasible solution' in solver_status:
        is_optimal = 'infisible'
    
    # Print results
    print(f'solver: {solver} - optimal: {is_optimal}, objective: {obj_value}, solve time: {solve_time}\n')
    
    # Extract paths and objects for each courier
    X = ampl.get_variable('X')
    paths = []
    courier_objects = []
    
    for i in range(1, instance['m'] + 1):
        path = []
        objects = []
        current_node = instance['n'] + 1  # Start at the origin (n+1)

        while True:
            next_node = None
            for j in range(1, instance['n'] + 2):
                if X[i, current_node, j].value() > 0.5:  # Check if the edge is in the path
                    next_node = j
                    break

            if next_node is None or next_node == instance['n'] + 1:  # If no valid next node or returned to depot
                if current_node != instance['n'] + 1:
                    path.append((current_node, instance['n'] + 1))  # End at the depot
                break

            path.append((current_node, next_node))
            if next_node <= instance['n']:  # If the next node is an object
                objects.append(next_node)
            current_node = next_node
        
        paths.append(path)
        courier_objects.append(objects)
    
    # Print the paths
    for i, path in enumerate(paths, 1):
        path_str = ' -> '.join(f'{x}->{y}' for x, y in path)
        print(f'Courier {i} path: {path_str}')
    
    print(paths)
    print(courier_objects)

    return solve_time, is_optimal, obj_value, courier_objects

In [10]:
def run_MIP_model_SB(instance, solver):
    ampl = AMPL(Environment())
    
    model_path = "MIP/model_SB.mod"
    ampl.read(model_path)

    ampl.param["m"] = instance['m']
    ampl.param["n"] = instance['n']
    ampl.param['l_values'] = instance['l_values']
    ampl.param['s_values'] = instance['s_values']
    ampl.param['distance_matrix'] = np.array(instance['distance_matrix'])
    
    # Set solver and time limit
    ampl.option["solver"] = solver
    ampl.option[f"{solver}_options"] = "timelim=300"
    
    # Solve the model
    ampl.solve()
    
    # Check if the solution is optimal
    is_solved = ampl.get_value("solve_result")
    obj_value = int(ampl.get_objective('obj_max_distance').value())
    solve_time = int(ampl.get_value('_total_solve_time'))
    solver_status = ampl.get_value('solve_message')
    
    # Determine if the solution is optimal
    if 'optimal solution' in solver_status:
        is_optimal = 'true'
    else:
        is_optimal = 'false'
    if 'infeasible solution' in solver_status:
        is_optimal = 'infisible'
    
    # Print results
    print(f'solver: {solver} - optimal: {is_optimal}, objective: {obj_value}, solve time: {solve_time}\n')
    
    # Extract paths and objects for each courier
    X = ampl.get_variable('X')
    paths = []
    courier_objects = []
    
    for i in range(1, instance['m'] + 1):
        path = []
        objects = []
        current_node = instance['n'] + 1  # Start at the origin (n+1)

        while True:
            next_node = None
            for j in range(1, instance['n'] + 2):
                if X[i, current_node, j].value() > 0.5:  # Check if the edge is in the path
                    next_node = j
                    break

            if next_node is None or next_node == instance['n'] + 1:  # If no valid next node or returned to depot
                if current_node != instance['n'] + 1:
                    path.append((current_node, instance['n'] + 1))  # End at the depot
                break

            path.append((current_node, next_node))
            if next_node <= instance['n']:  # If the next node is an object
                objects.append(next_node)
            current_node = next_node
        
        paths.append(path)
        courier_objects.append(objects)
    
    # Print the paths
    for i, path in enumerate(paths, 1):
        path_str = ' -> '.join(f'{x}->{y}' for x, y in path)
        print(f'Courier {i} path: {path_str}')
    
    print(paths)
    print(courier_objects)

    return solve_time, is_optimal, obj_value, courier_objects
      


In [11]:
j = 0
for filename, instance in instances.items():
    solvers_SB = [f'{solver}_symbreak' for solver in solvers]
    solvers_list = solvers+solvers_SB
    print(f'Solving {filename}:\n')
    solve_times = []
    optimality_list = []
    objs = []
    sols = []
    for solver in solvers:
        solve_time, is_optimal, obj_value, solution = run_MIP_model(instance, solver)
        solve_times.append(solve_time)
        optimality_list.append(is_optimal)
        objs.append(obj_value)
        sols.append(solution)
    for solver in solvers:
        solve_time, is_optimal, obj_value, solution = run_MIP_model_SB(instance, solver)
        solve_times.append(solve_time)
        optimality_list.append(is_optimal)
        objs.append(obj_value)
        sols.append(solution)
    
    data = {}
    for i in range(len(solvers_list)):
        print(sols[i])
        data[solvers_list[i]] = {
            'time': solve_times[i],
            'optimal': optimality_list[i],
            'obj': objs[i],
            'sol': sols[i]
        }
    j += 1
    print(data)
    with open(f'res/MIP/{j}.json', 'w') as f:
        json.dump(data, f, default=str)


Solving inst01.dat:

HiGHS 1.7.0:   lim:time = 300
HiGHS 1.7.0: optimal solution; objective 14
2937 simplex iterations
31 branching nodes
solver: highs - optimal: true, objective: 14, solve time: 0

Courier 1 path: 7->1 -> 1->3 -> 3->4 -> 4->7
Courier 2 path: 7->2 -> 2->5 -> 5->6 -> 6->7
[[(7, 1), (1, 3), (3, 4), (4, 7)], [(7, 2), (2, 5), (5, 6), (6, 7)]]
[[1, 3, 4], [2, 5, 6]]
SCIP 9.0.1:   lim:time = 300
SCIP 9.0.1: optimal solution; objective 14
802 simplex iterations
38 branching nodes
solver: scip - optimal: true, objective: 14, solve time: 0

Courier 1 path: 7->4 -> 4->3 -> 3->1 -> 1->7
Courier 2 path: 7->2 -> 2->5 -> 5->6 -> 6->7
[[(7, 4), (4, 3), (3, 1), (1, 7)], [(7, 2), (2, 5), (5, 6), (6, 7)]]
[[4, 3, 1], [2, 5, 6]]
HiGHS 1.7.0:   lim:time = 300
HiGHS 1.7.0: optimal solution; objective 14
386 simplex iterations
1 branching nodes
solver: highs - optimal: true, objective: 14, solve time: 0

Courier 1 path: 7->4 -> 4->3 -> 3->1 -> 1->7
Courier 2 path: 7->2 -> 2->5 -> 5->6 -> 6-